In [1]:
import pandas as pd
from itertools import product

In [2]:
# Файл с ранее согласованным планом продаж для каждого МП по каждому SKU
plan = pd.read_excel('ОПР.xlsx', sheet_name='План')
# Файл со списоком всех регионов РФ
regions = pd.read_excel('Регионы.xlsx')
# Справочник Регион-ФО
fo = pd.read_excel('Регионы.xlsx', sheet_name='Лист2')
# Справочник ТМ-Регион
tm = pd.read_excel('Регионы.xlsx', sheet_name='Лист3')
# Сопоставление регионов из двух разных источников (написаны по-разному)
reg_sorex = pd.read_excel('Регионы.xlsx', sheet_name='Лист4')
# Справочник Месяц-DFU-Цена
price = pd.read_excel('Регионы.xlsx', sheet_name='Лист5')

В данном направлении продаж (ревматология) медицинские представители (МП) присутствуют не во всех регионах страны. На этапе разработки методологии распределения планов было решено оставить только те регионы, в которых есть МП. Но в дальнейшем выяснилось, что если в файле с планами не указаны какие-то регионы, то факты продаж по этим регионам в отчет не попадают. 
В переменной all_combinations с помощью декартова произведения создается ДФ, в котором перемножаются все значения дат, регионов и DFU (уникальный код товара). 

In [3]:
regions = regions['Регион'].tolist()
dates = plan['Дата'].unique()
dfu = plan['DFU'].unique()
all_combinations = pd.DataFrame(product(dates, regions, dfu), columns=['Дата', 'Регион', 'DFU'])

In [4]:
dfu = plan[['Бренд', 'Наименование', 'DFU']].drop_duplicates()
fo_people = plan[['Регион', 'ТМ', 'МП']].drop_duplicates()

К таблице со столбцами Дата, Регион, DFU присоединяем справочники ФО, наименований препаратов, ФИО территориальных менеджеров (ТМ) и ФИО МП, тем самым создаем шаблон файла с планом продаж.

In [5]:
all_combinations = all_combinations.merge(dfu, how='left', on='DFU')

In [6]:
all_combinations = all_combinations.merge(fo_people, how='left', on='Регион')
all_combinations = all_combinations.merge(fo, how='left', on='Регион')
all_combinations = all_combinations.merge(tm, how='left', on= 'Регион', suffixes=('', '_new'))

In [7]:
all_combinations['ТМ'] = all_combinations['ТМ'].fillna(all_combinations['ТМ_new'])
all_combinations.drop(columns=['ТМ_new'], inplace=True)
all_combinations['МП'] = all_combinations['МП'].fillna('Нет МП')

К основной таблице с планами присоединяем наш шаблон и в столбце "План, уп" значения NaN заменяем на 0.

In [8]:
merge_plan = all_combinations.merge(plan, how='left', on=['Бренд', 'Наименование', 'DFU', 'ФО', 'Регион', 'ТМ', 'МП', 'Дата'])

In [9]:
merge_plan['План,уп']=merge_plan['План,уп'].fillna(0)

Создаем словарь с ФИО МП и датой увольнения. Словарь нужен для того, чтобы планы уволившегося МП переносить на "Вакансия", при этом сохраняя историчность.

In [10]:
mp_replacements = {
    'МП_1': '2025-02-01',
    'МП_2': '2025-03-01',
    'МП_3': '2025-03-01',
    'МП_4': '2025-03-01',
    'МП_5': '2025-03-01',
    'МП_6': '2025-04-01'
}

In [11]:
for mp, start_date in mp_replacements.items():
    regions_mp = merge_plan[merge_plan['МП'] == mp]['Регион'].unique()
    merge_plan.loc[(merge_plan['Дата'] >= start_date) & 
                   (merge_plan['Регион'].isin(regions_mp)) & 
                   (merge_plan['МП'] == mp), 'МП'] = 'Вакансия'

С апреля у ТМ_1 в дополнение к существующим МП появляется дополнительная вакансия. Нужно равномерно перераспределить планы в упаковках для всех МП и вакансии данного ТМ в их регионах. Историчность сохраняем.

In [12]:
merge_plan.loc[(merge_plan['Дата'] >= '2025-04-01') &
               (merge_plan['ТМ']=='ТМ_1') &
               (merge_plan['Регион'] == 'Калининградская область'),
                'МП'
]='Вакансия'

In [13]:
mp = merge_plan[(merge_plan['ТМ'] == 'ТМ_1') & (merge_plan['МП'] != 'Нет МП')][['ТМ', 'МП']]
mp = mp.drop_duplicates()

In [14]:
reg = ['Калининградская область', 'Санкт-Петербург', 'Ленинградская область']

In [15]:
vacancy = merge_plan.loc[(merge_plan['Дата'] >= '2025-04-01') &
               (merge_plan['ТМ']=='ТМ_1') &
               (merge_plan['Регион'].isin(reg))]

In [16]:
count_mp = vacancy['МП'].nunique()
vacancy = vacancy.groupby(by=['Дата','DFU','Регион', 'ТМ'], as_index=False)['План,уп'].sum()


In [18]:
vacancy['План 1_МП'] = vacancy['План,уп'] / count_mp

In [19]:
def user_round(x):
    if x - int(x) > 0.5:
        return int(x) + 1
    else:
        return int(x)
# Функция для округления планов в упаковках

Создаем дф по образу и подобию нашего действующего плана, но уже с перераспределенными упаковками для конкретного ТМ.

In [20]:
vacancy['План 1_МП'] = vacancy['План 1_МП'].map(user_round)

In [21]:
vacancy = vacancy.merge(fo, on = 'Регион', how = 'left')

In [22]:
vacancy = vacancy.merge(dfu, on='DFU', how='left')

In [23]:
vacancy = vacancy.merge(mp, on='ТМ', how='left')

In [24]:
vacancy.drop(columns=['План,уп'], inplace=True)

In [25]:
vacancy = vacancy.rename(columns={'План 1_МП': 'План,уп'})

Из основного файла с планами удаляем неактуальную информацию по ТМ_1.

In [26]:
merge_plan = merge_plan.loc[~((merge_plan['Дата'] >= '2025-04-01') &
                              (merge_plan['ТМ'] == 'ТМ_1') &
                              (merge_plan['Регион'].isin(reg)))]

К плану продаж добавляем актуальную информацию по ТМ_1.

In [27]:
merge_plan = pd.concat([merge_plan,vacancy],ignore_index=True)

Для удобства коллег заменяем регионы на другое написание.

In [28]:
merge_plan = merge_plan.merge(reg_sorex, how='left', on='Регион')
merge_plan.drop(columns=['Регион'], inplace=True)

Добавляем цены, переводим упаковки в рубли.

In [29]:
merge_plan = merge_plan.merge(price, how='left', on=['Дата', 'DFU'])

In [30]:
merge_plan['План,руб'] = merge_plan['План,уп']*merge_plan['Цена']

In [31]:
merge_plan.to_excel('План ОПР_26032025.xlsx')